In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import glob, os

## BoW Feature Extraction

### Step:

#### I. Building CodeBook
1. Using sift to detect keypoints and extract feature descriptors of all image in training set
2. Using K-means with number_of_cluster = 20.000

#### II. Compute histogram vector (a 20.000-dimention vector for each image)
1. With each input image, extract all feature descriptors
2. Assign each descriptor vector into the centers
3. make histogram of these

### About SIFT

### Split train sets and test sets 

In [1]:
import glob, os
from sklearn.model_selection import train_test_split

"""
# load feature
with open('features.pkl', 'rb') as feature_file:
    features = pickle.load(feature_file)

features = features.reshape((features.shape[0], features.shape[2]))
"""

# load all image name
images = glob.glob('dataset/101_ObjectCategories//*//*.jpg')
images.sort()
print(len(images))

# set labels for each image
targets = []
for image in images:
    targets.append(image.split("/")[1])

# split to train/test set
num_db = 3
for i in range(num_db):
    X_train, X_test, Y_train, Y_test = train_test_split(images, targets, test_size=0.2)
    db_path = 'db/db%s'%(i+1)
    if not os.path.exists(db_path):
        os.makedirs(db_path)

    with open(db_path+'/train.txt', mode='w') as train_file:
        for file in X_train:
            train_file.write(file)
            train_file.write('\n')

    with open(db_path+'/lbtrain.txt', mode='w') as label_train_file:
        for label in Y_train:
            label_train_file.write(label)
            label_train_file.write('\n')

    with open(db_path+'/test.txt', mode='w') as test_file:
        for file in X_test:
            test_file.write(file)
            test_file.write('\n')

    with open(db_path+'/lbtest.txt', mode='w') as label_test_file:
        for label in Y_test:
            label_test_file.write(label)
            label_test_file.write('\n')

print("Done")



9144
Done


#### import LIBs

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
# from LoadData import LoadData

from scipy.spatial import distance

%matplotlib inline

### Extract feature descriptors in trainning set

In [4]:
def noneMulticoreFeatureDescripting(img_paths_list):
	all_keypoints = []
	all_descriptors = np.array([], dtype=np.float32).reshape(0,128)
	sift = cv2.xfeatures2d.SIFT_create()
	i = 1
	for image_path in img_paths_list:
		print(str(i), ' --- Processing on:', image_path)
		i = i + 1
		image = cv2.imread(image_path)
		gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		# key_points = sift.detect(gray_img, None)
		key_points, descriptors = sift.detectAndCompute(gray_img, None)
		all_keypoints = all_keypoints + key_points
		# print('descriptors:\n', descriptors)
		# all_descriptors = all_descriptors + descriptors
		# all_descriptors.append(descriptors)
		# print (all_descriptors.shape)
		# print ( descriptors.shape)
		all_descriptors = np.concatenate((all_descriptors, descriptors), axis=0)
	return all_descriptors, all_keypoints

### Appying multicore

In [5]:
def featureExtraction(img_paths_list, start_point, end_point, queue):
	i = start_point
	sift = cv2.xfeatures2d.SIFT_create()        
	all_keypoints = []
	all_descriptors = []
	if start_point <= len(img_paths_list) :
		if end_point <= len(img_paths_list): 
			path_list = img_paths_list[start_point:end_point]
		else:
			path_list = img_paths_list[start_point:len(img_paths_list)]
	else:
		path_list = []

	for image_path in path_list:
		print(str(i),'  --  ',image_path)
		i = i + 1
		image = cv2.imread(image_path)
		gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		# key_points = sift.detect(gray_img, None)
		key_points, descriptors = sift.detectAndCompute(gray_img, None)
		all_keypoints = all_keypoints + key_points
		all_descriptors.append(descriptors)
	print("Finish process in 1 core!")
	queue.put(all_descriptors)

def multiprocessingFeatureDescripting():
	# multicores processing
	num_of_cores = 32
	distance = int(len(img['trainImage'])/num_of_cores)
	queue = Queue()
	similarities = [Process(target=featureExtraction, args=(img['trainImage'], x*distance, (x+1)*distance, queue)) for x in range (0,num_of_cores+1)]

	for similar in similarities:
		similar.start()

	result = []
	print('Appending in result!')
	for similar in similarities:
		similar.join(10)
		result.append(queue.get(True))

	return result

### Building Codeword

In [6]:
def buildingCodeWord():
	img_paths_list = img['trainImage']
	all_descriptor_vectors, all_keypoints = noneMulticoreFeatureDescripting(img_paths_list)
	print('Number of descriptors: ',len(all_descriptor_vectors))
	print(all_descriptor_vectors)
	print('Finish extracting descriptors!')
	# Number of clusters
	kmeans = KMeans(n_clusters=20000)
	# Fitting the input data
	kmeans = kmeans.fit(np.array(all_descriptor_vectors))

	# Getting the cluster labels
	labels = kmeans.predict(np.array(all_descriptor_vectors))
	# Centroid values
	centroids = kmeans.cluster_centers_
	print('Finish clustering')
	# Comparing with scikit-learn centroids
	print(centroids) # From sci-kit learn

	# Write to file:
	data_path = "codeword"
	if not os.path.exists(data_path):
		os.makedirs(data_path)
	codewords_file_path = data_path + '/codewords.dat'   
	with open(codewords_file_path, mode='wb') as codeword_file:
		# for centroid in centroids:
		# 	codeword_file.write(str(centroid))
		# 	codeword_file.write('\n')
		# np.save(codeword_file, centroids)
		pickle.dump(centroids, codeword_file)

	print("Finish building codebook!")

- Number of dimentions = number of cluster 

### Compute histogram

#### Compute histogram for an image

In [ ]:
def bow_feature_extraction(centroids, image):
    # inititalize sift detector
    sift = cv2.xfeatures2d.SIFT_create()
    
    # read image and extract sift features descriptor:
    image = cv2.imread(image_path)
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    key_points, descriptors = sift.detectAndCompute(gray_img, None)
    
    # Initialize the historgram ( ~ feature vector)
    # number_of_demention_of_histogram = len(centroids)
    histogram = np.zeros(number_of_demention_of_histogram)
    
    for keypoint_descriptor in keypoint_descriptors:
        print ('---------------')
        print ('des_vector: ', keypoint_descriptor)
        distances_array = []
        i = 0
        for centroid in centroids:
        #     distance = distance.euclidean(a,vector)
            # Euclidean computation
            distance = np.linalg.norm(np.array(keypoint_descriptor)-np.array(centroid))
            print(i, ',  ', distance)
            i += 1
        #     np.concatenate((distances_array, distance), axis=0)
        #     distances_array.append(distance)
            distances_array.append(distance)
        print ('- feature_vector_length:  ', len(distances_array))
        print('- distance array: ', distances_array)
        print ('minpost: %i'%distances_array.index(min(distances_array)))
        min_index = distances_array.index(min(distances_array))
        histogram[min_index] += 1
        print('histogram: ', histogram)
    print('feature vector under histogram form: ', histogram)
    return histogram

#### Compute all bow_feature_vector (histograms) from image set and add to list (for using purpose later with SVM classifier)

In [18]:
import _pickle as pickle
from LoadData import LoadData
# not finished yet
# # load file pickle
def compute_vectors_for_image_dataset:
    with open('codeword/codewords.dat', 'rb') as handle:
        centroids = pickle.load(handle)

    print(centroids)    
    print(len(centroids))
    all_descriptors = []

    # read image paths
    img = LoadData().getImagePath('db1')
    img_paths_list = [img['trainImage'][0],] # read all image path in train images set
    for image_path in img_paths_list:
        all_descriptors.append(bow_feature_extraction(centroids, image_path))

    return all_descriptors
    

[[ 44.18101501  14.23808861   7.3192091  ...,  18.30872917   3.90270853
    6.54583168]
 [ 26.20446396  28.44099808  30.0435009  ...,  20.65877533  22.01660156
   20.33893967]
 [ 16.62963295   7.91505909   8.70709991 ...,  10.57439423   9.37574768
   11.03869152]
 ..., 
 [ 26.34394073  24.1885891   24.1035881  ...,  14.76890945  15.89593887
   18.36178207]
 [ 18.51602364  20.86983299  25.46174431 ...,  11.50701809  15.48223877
   17.26696968]
 [ 15.38855553  12.05156708  12.68083763 ...,   5.28268433   4.06098938
   12.13243866]]
9


AttributeError: module 'cv2.cv2' has no attribute 'xfeatures2d'

### Evaluating feature vectors under Histogram form

- Number of dimentions = number of cluster 

In [3]:
keypoint_descriptor_1 = [1, 1, 1, 1]
keypoint_descriptor_2 = [1, 1, 1, 2]
keypoint_descriptor_3 = [2, 10, 7, 1]
keypoint_descriptor_4 = [8,9,7,1]
keypoint_descriptor_5 = [9, 1, 1, 5]
keypoint_descriptor_6 = [9, 1, 1, 9]


c1 = [1, 1, 2, 1]
c2 = [2,3,4,5]
c3 = [8,9,7,1]
c4 = [2,10,10,2]

In [6]:
centroids = []
centroids.append(c1)
centroids.append(c2)
centroids.append(c3)
centroids.append(c4)


keypoint_descriptors = []
keypoint_descriptors.append(keypoint_descriptor_1)
keypoint_descriptors.append(keypoint_descriptor_2)
keypoint_descriptors.append(keypoint_descriptor_3)
keypoint_descriptors.append(keypoint_descriptor_4)
keypoint_descriptors.append(keypoint_descriptor_5)
keypoint_descriptors.append(keypoint_descriptor_6)
# np.concatenate((d,b),sxi)


In [7]:
print(centroids)

[[1, 1, 2, 1], [2, 3, 4, 5], [8, 9, 7, 1], [2, 10, 10, 2]]


In [10]:
def bow_feature_extraction(centroids, image):
    # inititalize sift detector
    sift = cv2.xfeatures2d.SIFT_create()
    
    # read image and extract sift features descriptor:
    image = cv2.imread(image_path)
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    key_points, descriptors = sift.detectAndCompute(gray_img, None)
    
    # Initialize the historgram ( ~ feature vector)
    # number_of_demention_of_histogram = len(centroids)
    histogram = np.zeros(number_of_demention_of_histogram)
    
    for keypoint_descriptor in keypoint_descriptors:
        print ('---------------')
        print ('des_vector: ', keypoint_descriptor)
        distances_array = []
        i = 0
        for centroid in centroids:
        #     distance = distance.euclidean(a,vector)
            # Euclidean computation
            distance = np.linalg.norm(np.array(keypoint_descriptor)-np.array(centroid))
            print(i, ',  ', distance)
            i += 1
        #     np.concatenate((distances_array, distance), axis=0)
        #     distances_array.append(distance)
            distances_array.append(distance)
        print ('- feature_vector_length:  ', len(distances_array))
        print('- distance array: ', distances_array)
        print ('minpost: %i'%distances_array.index(min(distances_array)))
        min_index = distances_array.index(min(distances_array))
        histogram[min_index] += 1
        print('histogram: ', histogram)
    print('feature vector under histogram form: ', histogram)
    return histogram